In [2]:
import numpy as np


np.random.seed(42)


# Shape: (5 substations, 7 days, 4 time slots)

base_load = np.random.randint(80, 150, size=(5, 7, 4))


# Preview shape and data

print("Shape:", base_load.shape)

print("Sample (Substation 0):\n", base_load)

Shape: (5, 7, 4)
Sample (Substation 0):
 [[[131  94 140 100]
  [103  82 101 132]
  [ 81 109 117  81]
  [143 139 100 112]
  [137 101 128 138]
  [121 139  94 141]
  [141 126 141 130]]

 [[134 143  82 130]
  [ 86 100 118  97]
  [ 83 139  93  88]
  [132  81 139 123]
  [ 87 126 114 115]
  [129  83  81  85]
  [133  83 133 142]]

 [[ 97 123 113 141]
  [ 93 127  94 141]
  [119 132 103 105]
  [139 120 108  94]
  [124 144  88  80]
  [ 87 142  90  87]
  [114 114 112  84]]

 [[120 107  86  91]
  [113 112 127 102]
  [141 116 123 114]
  [144 126  82  80]
  [ 84  93 106  88]
  [ 94 121 130 142]
  [131  83 102  94]]

 [[122 108 115  92]
  [111 138 107 145]
  [121 124 141 136]
  [ 85 107 107 123]
  [109 141 141  80]
  [106 141  82 149]
  [106  88 141 116]]]


# 1. Apply Hourly Scaling Across Grid
Each of the 4 time blocks has a load factor due to ambient temperature differences.

In [3]:
load_factor = np.array([1.06, 0.48, 1.5, 0.60])

hourly_scaling = base_load * load_factor

print(hourly_scaling)
print(hourly_scaling.shape)

[[[138.86  45.12 210.    60.  ]
  [109.18  39.36 151.5   79.2 ]
  [ 85.86  52.32 175.5   48.6 ]
  [151.58  66.72 150.    67.2 ]
  [145.22  48.48 192.    82.8 ]
  [128.26  66.72 141.    84.6 ]
  [149.46  60.48 211.5   78.  ]]

 [[142.04  68.64 123.    78.  ]
  [ 91.16  48.   177.    58.2 ]
  [ 87.98  66.72 139.5   52.8 ]
  [139.92  38.88 208.5   73.8 ]
  [ 92.22  60.48 171.    69.  ]
  [136.74  39.84 121.5   51.  ]
  [140.98  39.84 199.5   85.2 ]]

 [[102.82  59.04 169.5   84.6 ]
  [ 98.58  60.96 141.    84.6 ]
  [126.14  63.36 154.5   63.  ]
  [147.34  57.6  162.    56.4 ]
  [131.44  69.12 132.    48.  ]
  [ 92.22  68.16 135.    52.2 ]
  [120.84  54.72 168.    50.4 ]]

 [[127.2   51.36 129.    54.6 ]
  [119.78  53.76 190.5   61.2 ]
  [149.46  55.68 184.5   68.4 ]
  [152.64  60.48 123.    48.  ]
  [ 89.04  44.64 159.    52.8 ]
  [ 99.64  58.08 195.    85.2 ]
  [138.86  39.84 153.    56.4 ]]

 [[129.32  51.84 172.5   55.2 ]
  [117.66  66.24 160.5   87.  ]
  [128.26  59.52 211.5   81.6 ]


# 2. Substation-Specific Policy Correction
Local incentives influence substations differently.

In [4]:
local_incentives_correction = np.array([4.2, 0.6, -2.3, 2.8, -0.5])

correctedPolicy = base_load + local_incentives_correction[:,np.newaxis,np.newaxis]
print(correctedPolicy)

[[[135.2  98.2 144.2 104.2]
  [107.2  86.2 105.2 136.2]
  [ 85.2 113.2 121.2  85.2]
  [147.2 143.2 104.2 116.2]
  [141.2 105.2 132.2 142.2]
  [125.2 143.2  98.2 145.2]
  [145.2 130.2 145.2 134.2]]

 [[134.6 143.6  82.6 130.6]
  [ 86.6 100.6 118.6  97.6]
  [ 83.6 139.6  93.6  88.6]
  [132.6  81.6 139.6 123.6]
  [ 87.6 126.6 114.6 115.6]
  [129.6  83.6  81.6  85.6]
  [133.6  83.6 133.6 142.6]]

 [[ 94.7 120.7 110.7 138.7]
  [ 90.7 124.7  91.7 138.7]
  [116.7 129.7 100.7 102.7]
  [136.7 117.7 105.7  91.7]
  [121.7 141.7  85.7  77.7]
  [ 84.7 139.7  87.7  84.7]
  [111.7 111.7 109.7  81.7]]

 [[122.8 109.8  88.8  93.8]
  [115.8 114.8 129.8 104.8]
  [143.8 118.8 125.8 116.8]
  [146.8 128.8  84.8  82.8]
  [ 86.8  95.8 108.8  90.8]
  [ 96.8 123.8 132.8 144.8]
  [133.8  85.8 104.8  96.8]]

 [[121.5 107.5 114.5  91.5]
  [110.5 137.5 106.5 144.5]
  [120.5 123.5 140.5 135.5]
  [ 84.5 106.5 106.5 122.5]
  [108.5 140.5 140.5  79.5]
  [105.5 140.5  81.5 148.5]
  [105.5  87.5 140.5 115.5]]]


# 3. Weekend Boost (Day 5 & 6)
Demand spikes on weekends by 15%

In [5]:
corrected_amounts = base_load[:,4:6,:] + (base_load[:,4:6,:] * 0.15)
print(corrected_amounts)

[[[157.55 116.15 147.2  158.7 ]
  [139.15 159.85 108.1  162.15]]

 [[100.05 144.9  131.1  132.25]
  [148.35  95.45  93.15  97.75]]

 [[142.6  165.6  101.2   92.  ]
  [100.05 163.3  103.5  100.05]]

 [[ 96.6  106.95 121.9  101.2 ]
  [108.1  139.15 149.5  163.3 ]]

 [[125.35 162.15 162.15  92.  ]
  [121.9  162.15  94.3  171.35]]]


# 4. Detect Overload Conditions (Threshold > 140 MW)
Flag any hourly reading that exceeds critical safe load.

In [6]:
condition = base_load > 140
print(condition)

[[[False False False False]
  [False False False False]
  [False False False False]
  [ True False False False]
  [False False False False]
  [False False False  True]
  [ True False  True False]]

 [[False  True False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False  True]]

 [[False False False  True]
  [False False False  True]
  [False False False False]
  [False False False False]
  [False  True False False]
  [False  True False False]
  [False False False False]]

 [[False False False False]
  [False False False False]
  [ True False False False]
  [ True False False False]
  [False False False False]
  [False False False  True]
  [False False False False]]

 [[False False False False]
  [False False False  True]
  [False False  True False]
  [False False False False]
  [False  True  True False]
  [False  True False  True]
  [False False  True False]]]


# 5. Multiply Time and Substation Factors Together
Combine operational risk (by time) with transformer degradation (by substation)

In [7]:
operational_risk = np.array([0.5, 1.04, 1.67, 0.84])
transformer_degradation = np.array([1.36, 0.9, 1.1, 2.3, 0.45])

result = base_load * transformer_degradation[:,np.newaxis,np.newaxis] * operational_risk[np.newaxis, np.newaxis, :]
print(result)

[[[ 89.08   132.9536 317.968  114.24  ]
  [ 70.04   115.9808 229.3912 150.7968]
  [ 55.08   154.1696 265.7304  92.5344]
  [ 97.24   196.6016 227.12   127.9488]
  [ 93.16   142.8544 290.7136 157.6512]
  [ 82.28   196.6016 213.4928 161.0784]
  [ 95.88   178.2144 320.2392 148.512 ]]

 [[ 60.3    133.848  123.246   98.28  ]
  [ 38.7     93.6    177.354   73.332 ]
  [ 37.35   130.104  139.779   66.528 ]
  [ 59.4     75.816  208.917   92.988 ]
  [ 39.15   117.936  171.342   86.94  ]
  [ 58.05    77.688  121.743   64.26  ]
  [ 59.85    77.688  199.899  107.352 ]]

 [[ 53.35   140.712  207.581  130.284 ]
  [ 51.15   145.288  172.678  130.284 ]
  [ 65.45   151.008  189.211   97.02  ]
  [ 76.45   137.28   198.396   86.856 ]
  [ 68.2    164.736  161.656   73.92  ]
  [ 47.85   162.448  165.33    80.388 ]
  [ 62.7    130.416  205.744   77.616 ]]

 [[138.     255.944  330.326  175.812 ]
  [129.95   267.904  487.807  197.064 ]
  [162.15   277.472  472.443  220.248 ]
  [165.6    301.392  314.962  154.

# 6. Normalize Each Substation's Weekly Profile (Max = 1.0)
For predictive comparison, scale each substation independently.

In [9]:
max_per_station = np.max(base_load,axis=(1,2),keepdims=True)
normalised_data = base_load / max_per_station
normalised_data

array([[[0.91608392, 0.65734266, 0.97902098, 0.6993007 ],
        [0.72027972, 0.57342657, 0.70629371, 0.92307692],
        [0.56643357, 0.76223776, 0.81818182, 0.56643357],
        [1.        , 0.97202797, 0.6993007 , 0.78321678],
        [0.95804196, 0.70629371, 0.8951049 , 0.96503497],
        [0.84615385, 0.97202797, 0.65734266, 0.98601399],
        [0.98601399, 0.88111888, 0.98601399, 0.90909091]],

       [[0.93706294, 1.        , 0.57342657, 0.90909091],
        [0.6013986 , 0.6993007 , 0.82517483, 0.67832168],
        [0.58041958, 0.97202797, 0.65034965, 0.61538462],
        [0.92307692, 0.56643357, 0.97202797, 0.86013986],
        [0.60839161, 0.88111888, 0.7972028 , 0.8041958 ],
        [0.9020979 , 0.58041958, 0.56643357, 0.59440559],
        [0.93006993, 0.58041958, 0.93006993, 0.99300699]],

       [[0.67361111, 0.85416667, 0.78472222, 0.97916667],
        [0.64583333, 0.88194444, 0.65277778, 0.97916667],
        [0.82638889, 0.91666667, 0.71527778, 0.72916667],
        [0

# 7. Daily Drop Simulation: Sudden Grid Curtailment
Simulate a 30% cut in evening slot (index 2) on day 3 only.

In [7]:
evening_slot = base_load[:,2,2] - (base_load[:,2,2] * 0.30)
print(evening_slot)

[81.9 65.1 72.1 86.1 98.7]


# 8. Apply Regional Baseline Offsets
Each substation has an offset based on elevation losses or legacy loads.
NumPy will broadcast offsets (5×1) → (5×7×4)

In [8]:
offset = np.linspace(0.8, 1.5, 5)
regional_baseline_offset = offset[:,np.newaxis,np.newaxis] + base_load
print(regional_baseline_offset)

[[[131.8    94.8   140.8   100.8  ]
  [103.8    82.8   101.8   132.8  ]
  [ 81.8   109.8   117.8    81.8  ]
  [143.8   139.8   100.8   112.8  ]
  [137.8   101.8   128.8   138.8  ]
  [121.8   139.8    94.8   141.8  ]
  [141.8   126.8   141.8   130.8  ]]

 [[134.975 143.975  82.975 130.975]
  [ 86.975 100.975 118.975  97.975]
  [ 83.975 139.975  93.975  88.975]
  [132.975  81.975 139.975 123.975]
  [ 87.975 126.975 114.975 115.975]
  [129.975  83.975  81.975  85.975]
  [133.975  83.975 133.975 142.975]]

 [[ 98.15  124.15  114.15  142.15 ]
  [ 94.15  128.15   95.15  142.15 ]
  [120.15  133.15  104.15  106.15 ]
  [140.15  121.15  109.15   95.15 ]
  [125.15  145.15   89.15   81.15 ]
  [ 88.15  143.15   91.15   88.15 ]
  [115.15  115.15  113.15   85.15 ]]

 [[121.325 108.325  87.325  92.325]
  [114.325 113.325 128.325 103.325]
  [142.325 117.325 124.325 115.325]
  [145.325 127.325  83.325  81.325]
  [ 85.325  94.325 107.325  89.325]
  [ 95.325 122.325 131.325 143.325]
  [132.325  84.325 103

# 9. Inverse Weekend Load Suppression
Flip the logic: weekdays normal, weekends(Day 5 & 6) reduced to 90%

In [9]:
weekends = base_load[:,4:6,:] * 0.9
print(weekends)

[[[123.3  90.9 115.2 124.2]
  [108.9 125.1  84.6 126.9]]

 [[ 78.3 113.4 102.6 103.5]
  [116.1  74.7  72.9  76.5]]

 [[111.6 129.6  79.2  72. ]
  [ 78.3 127.8  81.   78.3]]

 [[ 75.6  83.7  95.4  79.2]
  [ 84.6 108.9 117.  127.8]]

 [[ 98.1 126.9 126.9  72. ]
  [ 95.4 126.9  73.8 134.1]]]


# 10. Replicate Hourly Pattern to a Daily Cycle
One day's time pattern (e.g., [0.85, 1.0, 1.1, 0.95]) applied to every day.

In [10]:
time_pattern = np.array([0.85, 1.0, 1.1, 0.95])
daily_pattern = base_load * time_pattern[np.newaxis, np.newaxis, :]
print(daily_pattern)

[[[111.35  94.   154.    95.  ]
  [ 87.55  82.   111.1  125.4 ]
  [ 68.85 109.   128.7   76.95]
  [121.55 139.   110.   106.4 ]
  [116.45 101.   140.8  131.1 ]
  [102.85 139.   103.4  133.95]
  [119.85 126.   155.1  123.5 ]]

 [[113.9  143.    90.2  123.5 ]
  [ 73.1  100.   129.8   92.15]
  [ 70.55 139.   102.3   83.6 ]
  [112.2   81.   152.9  116.85]
  [ 73.95 126.   125.4  109.25]
  [109.65  83.    89.1   80.75]
  [113.05  83.   146.3  134.9 ]]

 [[ 82.45 123.   124.3  133.95]
  [ 79.05 127.   103.4  133.95]
  [101.15 132.   113.3   99.75]
  [118.15 120.   118.8   89.3 ]
  [105.4  144.    96.8   76.  ]
  [ 73.95 142.    99.    82.65]
  [ 96.9  114.   123.2   79.8 ]]

 [[102.   107.    94.6   86.45]
  [ 96.05 112.   139.7   96.9 ]
  [119.85 116.   135.3  108.3 ]
  [122.4  126.    90.2   76.  ]
  [ 71.4   93.   116.6   83.6 ]
  [ 79.9  121.   143.   134.9 ]
  [111.35  83.   112.2   89.3 ]]

 [[103.7  108.   126.5   87.4 ]
  [ 94.35 138.   117.7  137.75]
  [102.85 124.   155.1  129.2 ]


# 11. Multi-Axis Seasonal Scaling
Apply season-based multipliers across the week (7 days) for all substations and time blocks.
Broadcasts day-wise values across substations and time blocks.

In [11]:
seasonal_factors = np.random.rand(7)
seasonal_scaling = base_load * seasonal_factors[np.newaxis, :, np.newaxis]
print(seasonal_scaling)

[[[90.70912031 65.08898709 96.9410446  69.24360329]
  [27.74947038 22.09181137 27.21064571 35.56242806]
  [19.7741673  26.60968193 28.5626861  19.7741673 ]
  [24.06561903 23.39245486 16.82910422 18.84859672]
  [29.97069808 22.09518618 28.00182011 30.1894623 ]
  [67.53034224 77.57617828 52.46158819 78.69238228]
  [56.94090012 50.88335755 56.94090012 52.49870224]]

 [[92.78642841 99.0183527  56.7797547  90.01668428]
  [23.16946071 26.94123338 31.79065539 26.13299638]
  [20.26241835 33.93344759 22.70367357 21.48304596]
  [22.21441757 13.63157442 23.39245486 20.69979819]
  [19.0324871  27.56429167 24.93912103 25.15788525]
  [71.99515826 46.32246617 45.20626216 47.43867017]
  [53.71021075 33.5184022  53.71021075 57.34473629]]

 [[67.16629519 85.16963205 78.24527172 97.63348064]
  [25.05534704 34.21536639 25.32475938 37.98713907]
  [29.05093715 32.22456894 25.14492879 25.63317984]
  [23.39245486 20.19492506 18.17543255 15.81935796]
  [27.12676323 31.50204762 19.25125132 17.50113757]
  [48.55

# 12. Generate Load Penalty Zones
Apply a penalty (+5 MW) to all time slots where the load was below 100 MW.
Element-wise transformation using conditional broadcasting.

In [12]:
result = np.where(base_load < 100, base_load + 5, base_load)
print(result)

[[[131  99 140 100]
  [103  87 101 132]
  [ 86 109 117  86]
  [143 139 100 112]
  [137 101 128 138]
  [121 139  99 141]
  [141 126 141 130]]

 [[134 143  87 130]
  [ 91 100 118 102]
  [ 88 139  98  93]
  [132  86 139 123]
  [ 92 126 114 115]
  [129  88  86  90]
  [133  88 133 142]]

 [[102 123 113 141]
  [ 98 127  99 141]
  [119 132 103 105]
  [139 120 108  99]
  [124 144  93  85]
  [ 92 142  95  92]
  [114 114 112  89]]

 [[120 107  91  96]
  [113 112 127 102]
  [141 116 123 114]
  [144 126  87  85]
  [ 89  98 106  93]
  [ 99 121 130 142]
  [131  88 102  99]]

 [[122 108 115  97]
  [111 138 107 145]
  [121 124 141 136]
  [ 90 107 107 123]
  [109 141 141  85]
  [106 141  87 149]
  [106  93 141 116]]]


# 13. Apply Dynamic Correction Matrix
Simulate sensor recalibration where each substation-time combo has a fixed bias.
Broadcast across all days per substation + time block.

In [13]:
correction_matrix = np.random.rand(5, 4)
recalibration = base_load + correction_matrix[:, np.newaxis, :]
print(recalibration)

[[[131.06489225  94.25391541 140.24687606 100.69630427]
  [103.06489225  82.25391541 101.24687606 132.69630427]
  [ 81.06489225 109.25391541 117.24687606  81.69630427]
  [143.06489225 139.25391541 100.24687606 112.69630427]
  [137.06489225 101.25391541 128.24687606 138.69630427]
  [121.06489225 139.25391541  94.24687606 141.69630427]
  [141.06489225 126.25391541 141.24687606 130.69630427]]

 [[134.71227059 143.14808693  82.99774049 130.26678101]
  [ 86.71227059 100.14808693 118.99774049  97.26678101]
  [ 83.71227059 139.14808693  93.99774049  88.26678101]
  [132.71227059  81.14808693 139.99774049 123.26678101]
  [ 87.71227059 126.14808693 114.99774049 115.26678101]
  [129.71227059  83.14808693  81.99774049  85.26678101]
  [133.71227059  83.14808693 133.99774049 142.26678101]]

 [[ 97.97661496 123.41103701 113.03305073 141.34507125]
  [ 93.97661496 127.41103701  94.03305073 141.34507125]
  [119.97661496 132.41103701 103.03305073 105.34507125]
  [139.97661496 120.41103701 108.03305073  9

# 14. Differential Time-of-Day Priority Scaling
For a DR (demand response) simulation, penalize heavy use during noon and evening with a negative weight.
Promotes shifting load to cooler or off-peak blocks.

In [14]:
penalty = np.array([1.46, -0.5, -1.65, 0.89])

result = np.where(base_load > 100, base_load + penalty, base_load)
print(result)

[[[132.46  94.   138.35 100.  ]
  [104.46  82.    99.35 132.89]
  [ 81.   108.5  115.35  81.  ]
  [144.46 138.5  100.   112.89]
  [138.46 100.5  126.35 138.89]
  [122.46 138.5   94.   141.89]
  [142.46 125.5  139.35 130.89]]

 [[135.46 142.5   82.   130.89]
  [ 86.   100.   116.35  97.  ]
  [ 83.   138.5   93.    88.  ]
  [133.46  81.   137.35 123.89]
  [ 87.   125.5  112.35 115.89]
  [130.46  83.    81.    85.  ]
  [134.46  83.   131.35 142.89]]

 [[ 97.   122.5  111.35 141.89]
  [ 93.   126.5   94.   141.89]
  [120.46 131.5  101.35 105.89]
  [140.46 119.5  106.35  94.  ]
  [125.46 143.5   88.    80.  ]
  [ 87.   141.5   90.    87.  ]
  [115.46 113.5  110.35  84.  ]]

 [[121.46 106.5   86.    91.  ]
  [114.46 111.5  125.35 102.89]
  [142.46 115.5  121.35 114.89]
  [145.46 125.5   82.    80.  ]
  [ 84.    93.   104.35  88.  ]
  [ 94.   120.5  128.35 142.89]
  [132.46  83.   100.35  94.  ]]

 [[123.46 107.5  113.35  92.  ]
  [112.46 137.5  105.35 145.89]
  [122.46 123.5  139.35 136.89]


# 15. Ratio Matrix: Substation Utilization vs Peak

Compare every reading to its substation's maximum observed value.

In [39]:
n,m,l = base_load.shape
arr = []
ratio = np.zeros(base_load.shape)

for i in base_load:
    max_value = np.max(i)
    arr.append(max_value)
print(arr)

for i in range(n):
    for j in range(m):
        for k in range(l):
            ratio[i][j][k] = base_load[i][j][k] / arr[i]

print(ratio)


[143, 143, 144, 144, 149]
[[[0.91608392 0.65734266 0.97902098 0.6993007 ]
  [0.72027972 0.57342657 0.70629371 0.92307692]
  [0.56643357 0.76223776 0.81818182 0.56643357]
  [1.         0.97202797 0.6993007  0.78321678]
  [0.95804196 0.70629371 0.8951049  0.96503497]
  [0.84615385 0.97202797 0.65734266 0.98601399]
  [0.98601399 0.88111888 0.98601399 0.90909091]]

 [[0.93706294 1.         0.57342657 0.90909091]
  [0.6013986  0.6993007  0.82517483 0.67832168]
  [0.58041958 0.97202797 0.65034965 0.61538462]
  [0.92307692 0.56643357 0.97202797 0.86013986]
  [0.60839161 0.88111888 0.7972028  0.8041958 ]
  [0.9020979  0.58041958 0.56643357 0.59440559]
  [0.93006993 0.58041958 0.93006993 0.99300699]]

 [[0.67361111 0.85416667 0.78472222 0.97916667]
  [0.64583333 0.88194444 0.65277778 0.97916667]
  [0.82638889 0.91666667 0.71527778 0.72916667]
  [0.96527778 0.83333333 0.75       0.65277778]
  [0.86111111 1.         0.61111111 0.55555556]
  [0.60416667 0.98611111 0.625      0.60416667]
  [0.79166